# Gladier Tutorial
### Gladier: The Globus Architecture for Data-Intensive Experimental Research.

Gladier is a programmable data capture, storage, and analysis architecture for experimental facilities. The architecture leverages a data and computing substrate based on agents deployed across computer and storage systems at APS, ALCF, and elsewhere, all managed by cloud-hosted Globus services. In particular, we leverage [Globus Connect](https://www.globus.org/globus-connect)
and [funcX](https://funcx.org) agents to facilitate secure, reliable remote data and computation and employ the [Globus Flows](https://www.globus.org/platform/services/flows) platform to orchestrate distributed data management tasks into reliable pipelines.

## Gladier Toolkit
The Gladier toolkit provides tools and capabilities to simplify and accelerate the development of these automations. The toolkit manages the dynamic creation of flows, automatically registers funcX functions, and assists in validating inputs. 

Here we demonstrate how the Gladier toolkit can be used to let anyone create a simple, yet powerful client to automate data management tasks.

While not necessary to use this notebook, the Gladier toolkit is available on pypi and can be installed with:

    $ pip install gladier

Documentation is available [here.](https://gladier.readthedocs.io/en/latest/index.html)


In [ ]:
##General Imports
from pprint import pprint
import  os
import json

##Gladier Imports
from gladier import GladierBaseClient, GladierBaseTool, generate_flow_definition

##Globus automate variable for Binder
os.environ['SSH_TTY'] = 'BINDER_REMOTE'

## Creating the Client

In [ ]:
@generate_flow_definition
class Example_Client(GladierBaseClient):
    gladier_tools = [
        'gladier_tools.globus.Transfer', ##Note that the gladier client will import the tools by itself given a dotted path 
        'gladier_tools.posix.Tar',
    ]

exampleClient = Example_Client()


## General Variables
Those are the execution variables that will be used to create the payload for the flow

In [ ]:
##local inputs
localpath = 'c:\\'
myfile = 'awesome.sauce'
local_globus_endpoint = 'YOUR_ENDPOINT_UUID_GOES_HERE'

single = os.path.join(myfile) # make sure the file exists on your machine

# if not os.path.isfile(single):
#     with open(single,"w") as file1:
#         file1.writelines(["Gladier \n","Is \n","Awesome Sauce \n"])

##parse dirs
local_file = os.path.basename(single)
local_dir = os.path.dirname(single)

dest_file = ''
dest_dir = '/home/ubuntu/demo'

##Remote endpoints
dest_funcx_endpoint = '69378dfb-53e1-4ad4-adc9-ced8082730f9'
dest_globus_endpoint= '2e9e85d0-c4ad-11eb-87e2-559da91cd9a3'


## Flow Input
This is the payload that will be transfered at the beggining of the flow. The variable names should match the necessary variables inside the flow.

In [ ]:
base_input = {
    "input": {
        #Processing variables
        "transfer_source_path": os.path.join(local_dir,local_file),
        "transfer_destination_path": os.path.join(dest_dir,local_file),
        "tar_input": os.path.join(dest_dir,local_file),

        'transfer_recursive': False,

        # funcX endpoints
        "funcx_endpoint_compute": dest_funcx_endpoint,

        # globus endpoints
        "transfer_source_endpoint_id": local_globus_endpoint,
        "transfer_destination_endpoint_id": dest_globus_endpoint, 
    }
}

print(json.dumps(base_input, indent = 2)) ## notice this input can be used directly on automate website.

## Running the flow

In [ ]:
run_label = 'GladierTest_v2'

example_flow = exampleClient.run_flow(flow_input=base_input, label=run_label)
print("  File : " + base_input["input"]["transfer_source_path"])
print("  UUID : " + example_flow['action_id'])

## Monitoring the flow Progress

In [ ]:

exampleClient.progress(example_flow['action_id'])



## Getting the flow results 
We can access the results of each step of the run.


In [ ]:

exampleClient.get_details(example_flow['action_id'], 'Tar')


## Accessing the logs 


In [ ]:
print('https://app.globus.org/flows/%s/runs/%s' % (example_flow['flow_id'],example_flow['action_id']))